1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치
# %pip install -Uq docx2txt langchain_community langchain-text-splitters langchain_chroma langchain-pinecone pinecone


Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <archive url/path> ...

no such option: --upgrage


In [1]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고, 분할 ===========================================================
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200,
    length_function= len,
    is_separator_regex= False,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 =============================================
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 3. Pinecone vector DB =============================================================
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    documents= document_list,
    embedding=embedding,
    index_name='law-index',
)

# 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ================================
## 4.1 사용자 질문
query = '전세사기 피해자 금융지원에 대해 설명해주세요.'

## 4.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ============================
## 5.1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기 피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 5.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 5.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model = 'gpt-4o',)

## 5.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

c:\Users\1\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='전세사기 피해자 금융지원은 전세사기(전세금을 돌려받지 못하는 등)로 인해 경제적 어려움을 겪고 있는 피해자들을 지원하기 위한 정책 및 프로그램입니다. 일반적으로 정부와 금융기관이 협력하여 이러한 지원 프로그램을 운영합니다. 주요 지원 내용은 다음과 같습니다:\n\n1. **긴급 금융 지원**: 전세사기로 인해 급격히 자금이 필요한 피해자들에게 긴급 대출이나 생활자금을 지원합니다. 이자율이 낮고 상환 조건이 유연한 경우가 많습니다.\n\n2. **융자 및 대출 재조정**: 기존 대출의 상환 유예나 조건을 재조정하여 피해자가 금융 부담을 줄이도록 돕습니다.\n\n3. **임시 거주지 지원**: 거주하던 집을 잃은 피해자를 위해 임시 주거지를 제공하거나 주거비를 보조하는 프로그램을 운영합니다.\n\n4. **법률 및 상담 서비스**: 피해자들이 법적 조언을 받을 수 있도록 무료 상담을 제공하며, 필요한 경우 법적 대응을 지원합니다.\n\n5. **구제 및 복구 프로그램**: 피해자들이 손해를 최소화할 수 있도록 도와주며, 가능한 경우 손해를 복구하는 프로그램도 마련됩니다.\n\n6. **보험 및 보증 제도**: 전세사기에 대비한 보험 상품을 개발하거나 전세보증금을 보호하기 위한 보증 제도를 운영합니다.\n\n각 프로그램의 세부 사항은 정부 정책 및 지역에 따라 다를 수 있으며, 피해자들은 관련 기관이나 전문가를 통해 구체적인 지원 방법을 문의하는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 358, 'prompt_tokens': 58, 'total_tokens': 416, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction